In [44]:
import json 
import pandas as pd

In [72]:
f = open('results/clades.json',) 
  
# returns JSON object as  
# a dictionary 
data = json.load(f) 
x = data['nodes']
strain = []
for i in x:
    strain.append(i)  
clade = []
for y in x.values():
    for z in y.keys():
        if z == 'clade_membership':
            clade.append(y['clade_membership'])
list = [strain, clade]
df = pd.DataFrame(list, index =['strain', 'clade'])
df = df.T
drop = []
for i in df.index:
    if df.loc[i].str.startswith("NODE").values[0]:
        drop.append(i)
df = df.drop(drop)
df_meta = pd.read_csv('data/metadata.tsv', sep='\t')
df = pd.merge(df, df_meta, on="strain")
df

,strain,clade,virus,gisaid_epi_isl,genbank_accession,date,region,country,division,location,...,length,host,age,sex,originating_lab,submitting_lab,authors,url,title,date_submitted
0,Australia/NSW09/2020,A3,ncov,EPI_ISL_413595,?,2020-02-28,Oceania,Australia,New South Wales,Sydney,...,29409,Human,60,Female,Centre for Infectious Diseases and Microbiolog...,NSW Health Pathology - Institute of Clinical P...,Rockett et al,https://www.gisaid.org,?,2020-03-08
1,Australia/NSW114/2020,A2a,ncov,EPI_ISL_427724,?,2020-03-19,Oceania,Australia,New South Wales,Sydney,...,29415,Human,?,?,Centre for Infectious Diseases and Microbiolog...,NSW Health Pathology - Institute of Clinical P...,Lam et al,https://www.gisaid.org,?,2020-04-19
2,Australia/NSW142/2020,A2a,ncov,EPI_ISL_427749,?,2020-03-23,Oceania,Australia,New South Wales,Sydney,...,29415,Human,?,?,Centre for Infectious Diseases and Microbiolog...,NSW Health Pathology - Institute of Clinical P...,Bachmann et al,https://www.gisaid.org,?,2020-04-19
3,Australia/NSW153/2020,A2a,ncov,EPI_ISL_427653,?,2020-03-08,Oceania,Australia,New South Wales,Sydney,...,29415,Human,?,?,Centre for Infectious Diseases and Microbiolog...,NSW Health Pathology - Institute of Clinical P...,Rockett et al,https://www.gisaid.org,?,2020-04-19
4,Australia/NSW158/2020,A2a,ncov,EPI_ISL_427735,?,2020-03-18,Oceania,Australia,New South Wales,Sydney,...,29415,Human,?,?,Centre for Infectious Diseases and Microbiolog...,NSW Health Pathology - Institute of Clinical P...,Arnott et al,https://www.gisaid.org,?,2020-04-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,Wales/PHWC-2B1EE/2020,A2a,ncov,EPI_ISL_432272,?,2020-04-07,Europe,United Kingdom,Wales,NaN,...,29903,Human,?,?,Wales Specialist Virology Centre,Public Health Wales Microbiology Cardiff,Moore et al,https://www.gisaid.org,?,2020-04-29
393,Wuhan-Hu-1/2019,unassigned,ncov,EPI_ISL_402125,MN908947,2019-12-26,Asia,China,Hubei,Wuhan,...,29903,Human,?,?,unknown,National Institute for Communicable Disease Co...,Zhang et al,https://www.gisaid.org,?,2020-01-12
394,Wuhan/IPBCAMS-WH-04/2019,unassigned,ncov,EPI_ISL_403929,MT019532,2019-12-30,Asia,China,Hubei,Wuhan,...,29890,Human,52,Female,"Institute of Pathogen Biology, Chinese Academy...","Institute of Pathogen Biology, Chinese Academy...",Ren et al,https://www.gisaid.org,?,2020-01-21
395,Wuhan/WH01/2019,unassigned,ncov,EPI_ISL_406798,LR757998,2019-12-26,Asia,China,Hubei,Wuhan,...,29866,Human,44,Male,General Hospital of Central Theater Command of...,"BGI & Institute of Microbiology, Chinese Acade...",Chen et al,https://www.gisaid.org,?,2020-01-30


In [77]:
for i in df.clade.unique():
    print(i)
    print(df[df.clade == i].strain)

A3
0              Australia/NSW09/2020
12             Australia/NSW65/2020
32           Australia/VIC1233/2020
39           Australia/VIC1314/2020
74            Australia/VIC645/2020
78            Australia/VIC680/2020
98            Australia/VIC996/2020
161            Georgia/Tb-1523/2020
197                 India/1115/2020
243       Shandong/IVDC-SD-001/2020
251            Shanghai/SH0126/2020
375    UnitedArabEmirates/L068/2020
Name: strain, dtype: object
A2a
1      Australia/NSW114/2020
2      Australia/NSW142/2020
3      Australia/NSW153/2020
4      Australia/NSW158/2020
16       Australia/NT17/2020
               ...          
388    Wales/PHWC-275E6/2020
389    Wales/PHWC-27C69/2020
390    Wales/PHWC-281B4/2020
391    Wales/PHWC-29C94/2020
392    Wales/PHWC-2B1EE/2020
Name: strain, Length: 221, dtype: object
B
5                Australia/NSW181/2020
6                Australia/NSW184/2020
7                Australia/NSW193/2020
8                Australia/NSW200/2020
9              

Missing input file: None


In [78]:
df_try = df[df.clade == 'A3']
df_try.to_csv('A3list.tsv', header=False, index=False, sep='\t')

In [87]:
! python scripts/seq_select_by_id.py A3list.tsv 1 results/aligned.fasta FASTA out.fasta

Indexed 399 sequences
Selected 12 sequences by ID


In [88]:
from Bio import SeqIO
records = SeqIO.parse("out.fasta", "fasta")

In [89]:
from Bio import motifs

In [90]:
m = motifs.create([x.seq for x in records])
pwm = m.counts.normalize(pseudocounts=0.5)
pssm = pwm.log_odds()
matrix_draw = [pssm['A'],pssm['C'],pssm['G'],pssm['T']]

KeyError: 'N'